# EasyCV Object Detection - YOLOX

본문은 YOLOX 모델을 예로 들어, easyCV를 기반으로 목표 검출 모델을 훈련하고 예측하는 방법을 소개한다.

## 운영 환경 요구 사항

PAI-Pytorch 미러링 or 네이티브 Pytorch 1.5+ 이상 환경 GPU 기기, 메모리 32G 이상

## 설치 종속 패킷

비고: PAI-DSW docker에서 관련 의존도를 설치할 필요 없이 로컬 Notebook 환경에서 수행할 수 있습니다.

1. 먼저 pytorch 및 해당 버전의 torchvision을 설치하여 pytorch 1.5.1 이상 버전 지원

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd drive/MyDrive/Colab Notebooks/Conference

/content/drive/MyDrive/Colab Notebooks/Conference


In [5]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
pwd

'/content/drive/MyDrive/Colab Notebooks/Conference'

In [ ]:
import shutil
shutil.make_archive('Bbox_2_total', 'zip')

In [6]:
import gdown

google_path = 'https://drive.google.com/drive/u/0/folders/'
file_id = '1ApukgkiE8oLlYlDgly4zj9gfJt5oBN0A'
output_name = 'Bbox2_TOTAL.zip'

gdown.download(google_path+file_id, output_name, quiet=False)

/usr/local/lib/python3.7/dist-packages/gdown/parse_url.py:39: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  url="https://drive.google.com/uc?id={}".format(file_id)
Downloading...
From: https://drive.google.com/drive/u/0/folders/1ApukgkiE8oLlYlDgly4zj9gfJt5oBN0A
To: /content/drive/MyDrive/Colab Notebooks/Conference/Bbox2_TOTAL.zip
911kB [00:00, 55.2MB/s]


'Bbox2_TOTAL.zip'

In [ ]:
# install pytorch and torch vision
! pip install torch==1.10.0 torchvision==0.11.0

2. torch 및 cuda 버전 획득, 해당 버전의 mmcv 및 nvidia-dali 설치

In [ ]:
import torch
import os

os.environ['CUDA']='cu' + torch.version.cuda.replace('.', '')
os.environ['Torch']='torch'+torch.version.__version__.replace('+PAI', '')
!echo "cuda version: $CUDA"
!echo "pytorch version: $Torch"

cuda version: cu113
pytorch version: torch1.12.1+cu113


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/Colab Notebooks/Conference

/content/drive/MyDrive/Colab Notebooks/Conference


In [ ]:
# install some python deps

! pip install mmcv-full==1.4.4 -f https://download.openmmlab.com/mmcv/dist/${CUDA}/${Torch}/index.html
! pip install http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/third_party/nvidia_dali_cuda100-0.25.0-1535750-py3-none-manylinux2014_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.1+cu113/index.html
  Using cached mmcv-full-1.4.4.tar.gz (420 kB)
  ERROR: Failed building wheel for mmcv-full
  Running setup.py clean for mmcv-full
Failed to build mmcv-full
    Running setup.py install for mmcv-full ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-ayp5n6ih/mmcv-full_41237abab2814f3794ae541b3b4ef826/setup.py'"'"'; __file__='"'"'/tmp/pip-install-ayp5n6ih/mmcv-full_41237abab2814f3794ae541b3b4ef826/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip

3. EasyCV 알고리즘 패키지 설치

In [ ]:
!pip install pai-easycv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


4. 단순 검증

In [ ]:
from easycv.apis import *

## 이미지 탐지 모델 훈련 & 예측

### 데이터 준비

당신은 COCO 2017 데이터를 다운로드하거나 우리가 제공한 COCO 데이터를 사용할 수 있습니다.

In [ ]:
! wget http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/data/small_coco_demo/small_coco_demo.tar.gz && tar -zxf small_coco_demo.tar.gz

COCO 데이터 형식과 정확히 일치하도록 데이터 파일 이름 바꾸기

In [ ]:
!mkdir -p data/  && mv small_coco_demo data/coco

data/coco 형식은 다음과 같습니다.

```shell
data/coco/
├── annotations
│   ├── instances_train2017.json
│   └── instances_val2017.json
├── train2017
│   ├── 000000005802.jpg
│   ├── 000000060623.jpg
│   ├── 000000086408.jpg
│   ├── 000000118113.jpg
│   ├── 000000184613.jpg
│   ├── 000000193271.jpg
│   ├── 000000222564.jpg
│       ...
│   └── 000000574769.jpg
└── val2017
    ├── 000000006818.jpg
    ├── 000000017627.jpg
    ├── 000000037777.jpg
    ├── 000000087038.jpg
    ├── 000000174482.jpg
    ├── 000000181666.jpg
    ├── 000000184791.jpg
    ├── 000000252219.jpg
         ...
    └── 000000522713.jpg
```

### 모형 훈련

샘플 구성 파일 다운로드, YOLOX-S 모델 트레이닝 진행

In [ ]:
! rm -rf yolox_s_8xb16_300e_coco.py
! wget https://raw.githubusercontent.com/alibaba/EasyCV/master/configs/detection/yolox/yolox_s_8xb16_300e_coco.py

작은 데이터에 적응하기 위해서, 우리는 설정 파일 yolox_s_8xb16_300e_coco.py를 다음과 같은 필드를 수정하여, 훈련 epoch의 수를 줄이고, 로그 인쇄 빈도를 증가시킵니다.

```python

total_epochs = 3

#optimizer.lr -> 0.0002
optimizer = dict(
    type='SGD', lr=0.0002, momentum=0.9, weight_decay=5e-4, nesterov=True)

# log_config.interval 1
log_config = dict(interval=1)

```

주의: COCO 완전 데이터 트레이닝의 경우, 효과를 보장하기 위해 싱글 8 카드 트레이닝이 권장되며, 싱글 카드 트레이닝의 경우 학습률을 낮추는 것이 좋습니다. optimizer.lr

모델 효과를 보장하기 위해, 우리는 사전 훈련 모델을 기반으로 finetune을 수행하며, 다음과 같은 명령을 실행하여 훈련을 시작합니다.

In [ ]:
!python -m easycv.tools.train yolox_s_8xb16_300e_coco.py --work_dir work_dir/detection/yolox/yolox_s_8xb16_300e_coco --load_from http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/EasyCV/modelzoo/detection/yolox_s_bs16_lr002/epoch_300.pth

### 모델 내보내기
YOLOX 모델을 예측용으로 내보냅니다. 다음 명령을 실행하면 훈련에서 생성된 모델 파일을 볼 수 있습니다.

In [ ]:
! ls work_dir/detection/yolox/yolox_s_8xb16_300e_coco/*.pth

모델을 내보내기 전에 프로필을 수정하여 nms의 점수 임계값을 지정해야 합니다.

model.test_conf 0.01 -> 0.5

```python
model = dict(
    type='YOLOX',
    num_classes=80,
    model_type='s',  # s m l x tiny nano
    test_conf=0.5,
    nms_thre=0.65)
```

모델 내보내기를 위해 다음 명령을 실행합니다

In [ ]:
! cp yolox_s_8xb16_300e_coco.py yolox_s_8xb16_300e_coco_export.py && sed -i 's#test_conf=0.01#test_conf=0.5#g' yolox_s_8xb16_300e_coco_export.py
!python -m easycv.tools.export yolox_s_8xb16_300e_coco_export.py work_dir/detection/yolox/yolox_s_8xb16_300e_coco/epoch_30.pth  work_dir/detection/yolox/yolox_s_8xb16_300e_coco/yolox_export.pth

### 모형예측
테스트 그림 다운로드

In [ ]:
!wget http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/data/small_coco_demo/val2017/000000017627.jpg

In [ ]:
import cv2
from easycv.predictors import TorchYoloXPredictor

output_ckpt = 'work_dir/detection/yolox/yolox_s_8xb16_300e_coco/yolox_export.pth'
detector = TorchYoloXPredictor(output_ckpt)

img = cv2.imread('000000017627.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
output = detector.predict([img])
print(output)

In [ ]:
# view detection results

%matplotlib inline
from matplotlib import pyplot as plt
image = img.copy()
for box, cls_name in zip(output[0]['detection_boxes'], output[0]['detection_class_names']):
    # box is [x1,y1,x2,y2]
    box = [int(b) for b in box]
    image = cv2.rectangle(image, tuple(box[:2]), tuple(box[2:4]), (0,255,0), 2)
    cv2.putText(image, cls_name, (box[0], box[1]-5), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,255), 2)
plt.imshow(image)
plt.show()